# Ingeniería de atributos

En esta sección pasaremos a realizar las transformaciones de variables a partir de los patrones encontrados en el análisis exploratorio. Para ello, cargamos las librerías que venimos utilizando y el dataset completo. 

In [1]:
# cargamos las librerías
import pandas as pd 
import numpy as np  
import statsmodels.api as sm
from statsmodels.graphics import tsaplots
from statsmodels.graphics.tsaplots import plot_pacf
from datetime import datetime, timedelta 
import matplotlib.pyplot as plt
from matplotlib import style
import matplotlib as mpl
import seaborn as sns
from seaborn import distplot
import scipy as scp
# muteamos algunos warnings esperables que no afectan los outputs de los códigos
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
from google.colab import drive

In [2]:
aux = 'https://drive.google.com/file/d/1GWaKdMRk4Fx9qGgBMioNeO3E7_I3bA8k/view?usp=sharing'
link ='https://drive.google.com/uc?id=' + aux.split('/')[-2]
df = pd.read_csv(link)
df = df.set_index('Date')
df

,Volume USDT_btc,cryptocurrency_adjusted,crypto_adjusted,bitcoin_adjusted,bitcoin price_adjusted,ethereum_adjusted,ethereum price_adjusted,stock market_adjusted,wall street_adjusted,interest rate_adjusted,...,y_lag4,y_lag5,y_lag6,y_lag7,unix_btc,open_btc,high_btc,low_btc,close_btc,Volume BTC_btc
Date,,,,,,,,,,,,,,,,,,,,,
2015-08-08,3.801000,0.0,0.48,1.74,0.27,0.00,0.00,1.45,8.32,16.38,...,2.150000,1.825000,1.310000,1.645000,1438992000,275.920000,282.240000,262.650000,263.000000,1.032690e+03
2015-08-11,0.943500,0.0,0.88,1.89,0.49,0.00,0.00,2.61,9.92,19.74,...,1.825000,1.310000,1.645000,1.520000,1439251200,266.220000,269.260000,261.610000,269.260000,2.474900e+02
2015-08-12,2.558000,0.0,0.87,1.83,0.47,0.00,0.00,3.48,10.40,20.58,...,1.310000,1.645000,1.520000,1.410000,1439337600,269.260000,275.630000,263.330000,265.410000,6.821900e+02
2015-08-13,3.189000,0.0,0.87,1.95,0.35,0.00,0.00,2.61,10.40,18.48,...,1.645000,1.520000,1.410000,1.515000,1439424000,265.410000,269.880000,260.000000,263.000000,8.389900e+02
2015-08-14,2.129000,0.0,0.83,1.68,0.32,0.00,0.00,2.32,10.88,16.80,...,1.520000,1.410000,1.515000,1.615000,1439510400,263.000000,265.000000,255.000000,265.000000,5.562300e+02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-06-10,247.612626,5.0,6.46,8.41,8.40,9.10,7.56,18.00,6.48,61.00,...,1208.954979,1222.789134,1152.640234,1077.132870,1654819200,30093.109097,30363.965398,28863.371181,29091.044620,7.291809e+06
2022-06-11,177.801516,5.0,7.14,8.99,9.80,15.34,16.74,8.70,5.94,53.00,...,1222.789134,1152.640234,1077.132870,1041.031988,1654905600,29091.044620,29428.819243,28120.500000,28408.993958,5.092090e+06
2022-06-12,263.103277,7.2,8.16,11.02,11.20,15.34,15.39,7.20,6.03,46.00,...,1152.640234,1077.132870,1041.031988,1061.923798,1654992000,28423.198276,28528.145588,26578.603357,26600.000000,7.240280e+06


In [ ]:
df.columns

Index(['Date', 'Volume USDT_btc', 'cryptocurrency_adjusted', 'crypto_adjusted',
       'bitcoin_adjusted', 'bitcoin price_adjusted', 'ethereum_adjusted',
       'ethereum price_adjusted', 'stock market_adjusted',
       'wall street_adjusted', 'interest rate_adjusted', 'fed_adjusted',
       'bankruptcy_adjusted', 'china_adjusted', 'united states_adjusted',
       'war_adjusted', 'russia_adjusted', 'cryptocurrency_top_adjusted',
       'bitcoin_top_adjusted', 'investment_top_adjusted',
       'ethereum_top_adjusted', 'exchange_top_adjusted',
       'bankrup_top_adjusted', 'stock_market_top_adjusted',
       'inflation_adjusted', 'taxes_adjusted', 'digital_wallet_top_adjusted',
       'covid19_adjusted', 'pandemic_adjusted', 'elon musk_adjusted',
       'do kwon_adjusted', 'future_commodities_adjusted', 'banking_adjusted',
       'foreign_currency_adjusted', 'unix_eth', 'open_eth', 'high_eth',
       'low_eth', 'close_eth', 'Volume ETH_eth', 'Volume USDT_eth',
       'eth_close_open_mea

## Target: precios futuros de ethereum

En primer lugar, transformamos los siete precios futuros de ethereum calculando su logaritmo de base 10. No obstante, para evitar valores nulos que generen un valor Infinito como resultado, realizamos previamente le sumamos 1 a todos los valores de las series. Esta transformación es monótona y lineal, por lo que no afectarían a los resultados finales, solamente generará que los valores nulo de las series del target sigan siendo nulos.

In [3]:
# armado de variable target y conjunto de variables predictoras 
for i in range(1,8):
  df[f'log_y_lag{i}'] = np.log10(df[f'y_lag{i}']+1)
  df = df.drop(columns=[f'y_lag{i}'])
df

,Volume USDT_btc,cryptocurrency_adjusted,crypto_adjusted,bitcoin_adjusted,bitcoin price_adjusted,ethereum_adjusted,ethereum price_adjusted,stock market_adjusted,wall street_adjusted,interest rate_adjusted,...,low_btc,close_btc,Volume BTC_btc,log_y_lag1,log_y_lag2,log_y_lag3,log_y_lag4,log_y_lag5,log_y_lag6,log_y_lag7
Date,,,,,,,,,,,,,,,,,,,,,
2015-08-08,3.801000,0.0,0.48,1.74,0.27,0.00,0.00,1.45,8.32,16.38,...,262.650000,263.000000,1.032690e+03,0.352183,0.350248,0.441695,0.498311,0.451018,0.363612,0.422426
2015-08-11,0.943500,0.0,0.88,1.89,0.49,0.00,0.00,2.61,9.92,19.74,...,261.610000,269.260000,2.474900e+02,0.350248,0.441695,0.498311,0.451018,0.363612,0.422426,0.401401
2015-08-12,2.558000,0.0,0.87,1.83,0.47,0.00,0.00,3.48,10.40,20.58,...,263.330000,265.410000,6.821900e+02,0.441695,0.498311,0.451018,0.363612,0.422426,0.401401,0.382017
2015-08-13,3.189000,0.0,0.87,1.95,0.35,0.00,0.00,2.61,10.40,18.48,...,260.000000,263.000000,8.389900e+02,0.498311,0.451018,0.363612,0.422426,0.401401,0.382017,0.400538
2015-08-14,2.129000,0.0,0.83,1.68,0.32,0.00,0.00,2.32,10.88,16.80,...,255.000000,265.000000,5.562300e+02,0.451018,0.363612,0.422426,0.401401,0.382017,0.400538,0.417472
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-06-10,247.612626,5.0,6.46,8.41,8.40,9.10,7.56,18.00,6.48,61.00,...,28863.371181,29091.044620,7.291809e+06,3.203747,3.171949,3.121721,3.082769,3.087707,3.062070,3.032672
2022-06-11,177.801516,5.0,7.14,8.99,9.80,15.34,16.74,8.70,5.94,53.00,...,28120.500000,28408.993958,5.092090e+06,3.171949,3.121721,3.082769,3.087707,3.062070,3.032672,3.017881
2022-06-12,263.103277,7.2,8.16,11.02,11.20,15.34,15.39,7.20,6.03,46.00,...,26578.603357,26600.000000,7.240280e+06,3.121721,3.082769,3.087707,3.062070,3.032672,3.017881,3.026502


## Predictores

### Rezagos del precio de ethereum y bitcoin

En el caso de los predictores primero trabajamos sobre los precios presentes de bitcoin y ethereum. Aquí de nuevo realizamos una transformación del mismo tipo que con los targets (logaritmo base 10) con una transformación monónota previa (sumar 1). Luego, computamos los 7 rezagos del precio del bitcoin, que encontramos significativo a partir del test de Granger realizada en el análisis univariado. A su vez, los 2 primeros rezagos del precio de ethereum vistos en el análisis de autocorrelación parcial del mismo análisis. 

In [4]:
# ethereum
df['log_price_eth'] = np.log10(df['close_eth']+1)
# bitcoin
df['log_price_btc'] = np.log10(df['close_btc']+1)

In [5]:
# calculamos los rezagos de los 7 días anteriores al momento presente del precio de bitcoin 
for i in range(1,8):
  df[f'log_price_btc_lag{i}'] = df['log_price_btc'].shift(i)
# calculamos los rezagos de los 2 días anteriores al momento presente del precio de bitcoin 
for i in range(1,3):
  df[f'log_price_eth_lag{i}'] = df['log_price_eth'].shift(i)

In [6]:
# visualizamos las nuevas variables creadas
df[['log_price_btc','log_price_btc_lag1','log_price_btc_lag2', 
    'log_price_btc_lag3', 'log_price_btc_lag4','log_price_btc_lag5',
    'log_price_btc_lag6','log_price_btc_lag7']]

,log_price_btc,log_price_btc_lag1,log_price_btc_lag2,log_price_btc_lag3,log_price_btc_lag4,log_price_btc_lag5,log_price_btc_lag6,log_price_btc_lag7
Date,,,,,,,,
2015-08-08,2.421604,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-08-11,2.431782,2.421604,NaN,NaN,NaN,NaN,NaN,NaN
2015-08-12,2.425551,2.431782,2.421604,NaN,NaN,NaN,NaN,NaN
2015-08-13,2.421604,2.425551,2.431782,2.421604,NaN,NaN,NaN,NaN
2015-08-14,2.424882,2.421604,2.425551,2.431782,2.421604,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
2022-06-10,4.463774,4.478630,4.480211,4.493176,4.496540,4.475867,4.475200,4.472764
2022-06-11,4.453471,4.463774,4.478630,4.480211,4.493176,4.496540,4.475867,4.475200
2022-06-12,4.424898,4.453471,4.463774,4.478630,4.480211,4.493176,4.496540,4.475867


In [7]:
# visualizamos las nuevas variables creadas
df[['log_price_eth','log_price_eth_lag1','log_price_eth_lag2']]

,log_price_eth,log_price_eth_lag1,log_price_eth_lag2
Date,,,
2015-08-08,0.380211,NaN,NaN
2015-08-11,0.322219,0.380211,NaN
2015-08-12,0.376577,0.322219,0.380211
2015-08-13,0.498311,0.376577,0.322219
2015-08-14,0.498311,0.498311,0.376577
...,...,...,...
2022-06-10,3.221142,3.252827,3.253732
2022-06-11,3.185397,3.221142,3.252827
2022-06-12,3.157361,3.185397,3.221142


### Volumen de transacciones

En el caso del volumen de transacciones de ambas criptmonedas utilizadas, también realizamos una transformación logarítmica con el mismo procedimiento realizado antes, de manera de mantener valores más comprimidos y evitar que grandes saltos de magnitud generen problemas a los modelos. 

In [8]:
df['log_volume_btc-usd'] = np.log10(df['Volume USDT_btc']+1)
df['log_volume_eth-usd'] = np.log10(df['Volume USDT_eth']+1)

### Variación interdiaria de precios 

Si bien hemos visto que no existían efectos indirectos de la variación interdiaria del precio del bitcoin en la relación entre el precio presente de ethereum y su valor futuro, creemos conveniente agregar al análisis para contemplar la posibilidad de que exista un patrón directo que podría ser identificado por parte de los modelos. Como también, creamos la variación interdiaria del precio de ethereum por el mismo motivo.

In [9]:
# ethereum
df['var_price_eth'] = df['close_eth'].div(df['close_eth'].shift(1))-1
# bitcoin
df['var_price_btc'] = df['close_btc'].div(df['close_btc'].shift(1))-1

### Shocks pandemia y listado de ethereum 

Otra de las variables que debemos agregar en el dataset son los dos shocks relevantes que vimos en el análisis multivariado: la aparición del coronavirus y el listado de la criptomoneda ethereum en la plataforma de binance. Como se analizó antes, cada uno de estos shocks resultó tener un efecto indirecto importante al evaluar la relación entre el precio futuro y presente de ethereum, como también, entre predictores como el precio de bitcoin y nuestro target.

In [10]:
# shock covid-19
df.loc[df.index < '2020-03-12', 'Shock-COVID'] = 0
df.loc[df.index >= '2020-03-12', 'Shock-COVID'] = 1
# listado en binance
df.loc[df.index < '2017-08-17', 'Binance-Listing'] = 0
df.loc[df.index >= '2017-08-17', 'Binance-Listing'] = 1

### Términos de búsqueda

Por último, realizamos las mismas transformaciones logarítmicas que venimos realizan a los términos de búsqueda de google. La razón es similar al volumen de transacciones: el valor en logaritmo lograría una serie con valores más comprimidos, lo cual ayudaría a la hora de realizar las estimaciones.

In [11]:
list_gtrends = df.columns[(['adjusted' in i for i in df.columns])]
for y in list_gtrends:
  df[f'{y}_log'] = np.log10(df[f'{y}']+1)
df

,Volume USDT_btc,cryptocurrency_adjusted,crypto_adjusted,bitcoin_adjusted,bitcoin price_adjusted,ethereum_adjusted,ethereum price_adjusted,stock market_adjusted,wall street_adjusted,interest rate_adjusted,...,inflation_adjusted_log,taxes_adjusted_log,digital_wallet_top_adjusted_log,covid19_adjusted_log,pandemic_adjusted_log,elon musk_adjusted_log,do kwon_adjusted_log,future_commodities_adjusted_log,banking_adjusted_log,foreign_currency_adjusted_log
Date,,,,,,,,,,,,,,,,,,,,,
2015-08-08,3.801000,0.0,0.48,1.74,0.27,0.00,0.00,1.45,8.32,16.38,...,1.129690,1.486855,1.093071,0.000000,0.152288,0.235528,1.000000,1.387390,1.756788,1.414305
2015-08-11,0.943500,0.0,0.88,1.89,0.49,0.00,0.00,2.61,9.92,19.74,...,1.256237,1.642761,1.188928,0.000000,0.214844,0.257679,0.991226,1.499687,1.913602,1.584783
2015-08-12,2.558000,0.0,0.87,1.83,0.47,0.00,0.00,3.48,10.40,20.58,...,1.300161,1.610128,1.164353,0.000000,0.193125,0.220108,0.963788,1.517855,1.878062,1.613207
2015-08-13,3.189000,0.0,0.87,1.95,0.35,0.00,0.00,2.61,10.40,18.48,...,1.256237,1.604442,1.110590,0.000000,0.201397,0.257679,0.857332,1.523746,1.888516,1.584783
2015-08-14,2.129000,0.0,0.83,1.68,0.32,0.00,0.00,2.32,10.88,16.80,...,1.213783,1.580925,1.110590,0.000000,0.167317,0.278754,0.897627,1.480725,1.898725,1.548021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-06-10,247.612626,5.0,6.46,8.41,8.40,9.10,7.56,18.00,6.48,61.00,...,2.004321,1.522966,1.547775,0.738781,0.691965,0.477121,0.278754,1.707996,1.859978,1.937016
2022-06-11,177.801516,5.0,7.14,8.99,9.80,15.34,16.74,8.70,5.94,53.00,...,1.732394,1.382017,1.547775,0.704151,0.632457,0.477121,0.278754,1.596597,1.708931,1.824126
2022-06-12,263.103277,7.2,8.16,11.02,11.20,15.34,15.39,7.20,6.03,46.00,...,1.672098,1.357554,1.502427,0.691965,0.673021,0.531479,0.278754,1.561340,1.631444,1.781037


Un tipo de transformación que no realizamos es la separación de tendencia y ciclo de las series aproximadamente estacionarias. Creemos que trabajar con las componentes por separado puede producir un resultado positivo en las predicciones. No obstante, su inclusión será dejada para versiones futuras del trabajo.

Una vez tranformadas todas las variables reducimos el dataset al subconjunto de columnas relevantes. Debido a que todos los predictores fueron transformados, las mismas se ubican en el último segmento de variables en el orden de las columnas del dataset, por lo que al identificar la primera de ese orden (*log_y_lag1*) filtramos desde esa posición hasta el final.

In [12]:
# identificamos las columnas de interés y las filtramos
colIndex = df.columns.get_loc("log_y_lag1")
df = df.iloc[:, colIndex:]
# eliminamos todos los NA que aparecen a partir de las nuevas transformaciones
df = df.dropna()
df

,log_y_lag1,log_y_lag2,log_y_lag3,log_y_lag4,log_y_lag5,log_y_lag6,log_y_lag7,log_price_eth,log_price_btc,log_price_btc_lag1,...,inflation_adjusted_log,taxes_adjusted_log,digital_wallet_top_adjusted_log,covid19_adjusted_log,pandemic_adjusted_log,elon musk_adjusted_log,do kwon_adjusted_log,future_commodities_adjusted_log,banking_adjusted_log,foreign_currency_adjusted_log
Date,,,,,,,,,,,,,,,,,,,,,
2015-08-17,0.401401,0.382017,0.400538,0.417472,0.382917,0.382917,0.380211,0.401401,2.420566,2.411216,...,1.232488,1.632153,1.087071,0.000000,0.232996,0.212188,0.886491,1.474216,1.908699,1.554368
2015-08-18,0.382017,0.400538,0.417472,0.382917,0.382917,0.380211,0.337459,0.401401,2.401401,2.420566,...,1.278754,1.621280,1.028978,0.000000,0.206826,0.204120,0.991226,1.467608,1.888516,1.566791
2015-08-19,0.400538,0.417472,0.382917,0.382917,0.380211,0.337459,0.306425,0.361728,2.357935,2.401401,...,1.273233,1.632153,1.000434,0.000000,0.222716,0.243038,0.939519,1.493458,1.878062,1.578868
2015-08-20,0.417472,0.382917,0.382917,0.380211,0.337459,0.306425,0.329398,0.436163,2.371068,2.357935,...,1.261976,1.626751,1.121888,0.000000,0.184691,0.257679,1.021189,1.511883,1.883321,1.629410
2015-08-21,0.382917,0.382917,0.380211,0.337459,0.306425,0.329398,0.335458,0.397940,2.369457,2.371068,...,1.244525,1.586925,1.028978,0.000000,0.212188,0.278754,0.886491,1.546543,1.883321,1.607669
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-06-10,3.203747,3.171949,3.121721,3.082769,3.087707,3.062070,3.032672,3.221142,4.463774,4.478630,...,2.004321,1.522966,1.547775,0.738781,0.691965,0.477121,0.278754,1.707996,1.859978,1.937016
2022-06-11,3.171949,3.121721,3.082769,3.087707,3.062070,3.032672,3.017881,3.185397,4.453471,4.463774,...,1.732394,1.382017,1.547775,0.704151,0.632457,0.477121,0.278754,1.596597,1.708931,1.824126
2022-06-12,3.121721,3.082769,3.087707,3.062070,3.032672,3.017881,3.026502,3.157361,4.424898,4.453471,...,1.672098,1.357554,1.502427,0.691965,0.673021,0.531479,0.278754,1.561340,1.631444,1.781037


In [13]:
# guardar el dataset en carpeta data set del proyecto en google drive
drive.mount('drive')
df.to_csv('dataset.csv')
!cp dataset.csv "/content/drive/MyDrive/ds/proyecto/dataset/"

Mounted at drive
